## Chat myPDF : RAG with LangChain, Ollama, and FAISS Vector Store

In [1]:
!pip install --quiet langchain-community faiss-cpu langchain-huggingface  google-generativeai langchain-google-genai gradio jq langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 

In [2]:
# install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
# pull embedings
!nohup ollama serve &
!ollama pull  nomic-embed-text
!ollama pull llama3.2:3b
#!ollama pull


nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 970aa74c0a90:   2% ▕▏ 6.0 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:  12% ▕▏  32 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:  24% ▕▏  66 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:  30% ▕▏  81 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:  42% ▕▏ 114 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90:  47% ▕▏ 129 MB/274 MB                  pulling mani

In [4]:
! ollama list

NAME                       ID              SIZE      MODIFIED       
llama3.2:3b                a80c4f17acd5    2.0 GB    28 seconds ago    
nomic-embed-text:latest    0a109f422b47    274 MB    57 seconds ago    


In [5]:
import os
import warnings
import pandas as pd
import json
from langchain.embeddings import OllamaEmbeddings

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")



### Document Loader JSON

In [6]:
from langchain_community.document_loaders import JSONLoader

jsons = []
for root, dirs, files in os.walk('/content/sample_data/json-dataset'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.json'):
            jsons.append(os.path.join(root, file))

In [7]:
docs = []
for json in jsons:
    loader = JSONLoader(json,jq_schema=".[]",  # El archivo ya es una lista, así que simplemente tomamos cada elemento
                            text_content=False
                            )
    pages = loader.load()

    docs.extend(pages)

### Document Chunking

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

chunks = text_splitter.split_documents(docs)

In [9]:
len(docs), len(chunks)

(100, 1243)

In [10]:
len(docs[0].page_content), len(chunks[0].page_content)

(978, 90)

### Document Vector Embedding

In [12]:
import google.generativeai as genai
from google.colab import userdata
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain_ollama import ChatOllama
from google.colab import userdata

In [13]:
#Configure the genai library with the API key
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

#model = genai.GenerativeModel('gemini-2.0-flash')
#redhat demos
model = ChatOllama(model="llama3.2:3b", base_url="localhost:11434")

#testing model gemini
#response = model.("hola")

#testing model ollama
response = model.invoke("hola")

print(response.text)

<bound method BaseMessage.text of AIMessage(content='Hola! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-05-14T13:32:57.703723955Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2690776631, 'load_duration': 2125199547, 'prompt_eval_count': 27, 'prompt_eval_duration': 248246270, 'eval_count': 11, 'eval_duration': 314862234, 'model_name': 'llama3.2:3b'}, id='run--a2f9ec97-3bad-4f9d-a21a-ebe94bb87452-0', usage_metadata={'input_tokens': 27, 'output_tokens': 11, 'total_tokens': 38})>


In [14]:
#google embeddings

#embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=api_key)
#embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

#open ai embeddings
#embeddings = OpenAIEmbeddings(openai_api_key= userdata.get('OPENAI_API_KEY'))

#ollama embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

single_vector = embeddings.embed_query("this is some text data")
print(single_vector)


[0.454813152551651, 0.19418159127235413, -4.349863052368164, -1.0913795232772827, 0.6552602648735046, -0.3323252499103546, 0.3502291440963745, -0.3107079267501831, -0.23622243106365204, -0.6102532744407654, 0.08454950898885727, 0.4887739419937134, 1.5923538208007812, -0.04102497547864914, -0.9690878987312317, -0.8161699771881104, -1.1116968393325806, -1.0106348991394043, -0.31115955114364624, 1.666359782218933, 0.09680284559726715, 0.8051565885543823, -1.1879469156265259, -0.686189591884613, 1.2517915964126587, 0.7687667012214661, -0.4698296785354614, 0.24744176864624023, -1.7013026475906372, 0.22901788353919983, 0.7190706133842468, -0.6144566535949707, -0.35314807295799255, -0.3493945300579071, -1.2201651334762573, -0.6508439183235168, -0.2897360622882843, 0.31625935435295105, 0.5861876010894775, 0.6032562255859375, 1.0783110857009888, -0.8951320052146912, -0.16351671516895294, -0.878196120262146, 0.7127103805541992, -0.03746682405471802, 0.31323036551475525, 0.23912939429283142, 0.26

In [15]:
len(single_vector)

768

In [16]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [17]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [18]:
len(chunks)

1243

In [ ]:
# help(vector_store)

In [19]:
ids = vector_store.add_documents(documents=chunks)

In [20]:
vector_store.index_to_docstore_id
len(ids)

1243

In [21]:
# # # store vector database
db_name = "incidentes-facturas-db"
vector_store.save_local(db_name)

# # # # load vector database
# new_vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)
# len(new_vector_store.index_to_docstore_id)

### Retreival

In [22]:
question = "incidente 108"
parts = vector_store.search(query=question, search_type='similarity')

for doc in parts:
    print(doc.page_content)
    print("\n\n")

{"id_incidente": 108, "asunto": "Error de autenticaci\u00f3n con proveedor de firma", "creador":



{"id_incidente": 111, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 101, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 121, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":





In [23]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs = {'k': 10,
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})

In [24]:
partes_retriever = retriever.invoke(question)

for doc in partes_retriever:
     print(doc.page_content)
     print("\n\n")


{"id_incidente": 108, "asunto": "Error de autenticaci\u00f3n con proveedor de firma", "creador":



{"id_incidente": 111, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 101, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 121, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 103, "asunto": "Fallo en la validaci\u00f3n del RUC emisor", "creador": "Eduardo",



{"id_incidente": 118, "asunto": "Error de autenticaci\u00f3n con proveedor de firma", "creador":



{"id_incidente": 171, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 110, "asunto": "Error de formato en XML de e-Ticket", "creador": "Gabriel",



{"id_incidente": 191, "asunto": "Error al firmar comprobante electr\u00f3nico", "creador":



{"id_incidente": 138, "asunto": "Error de autenticaci\u00f3n con proveedor de firma", "creador":





In [25]:


# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"
question = "Incidente 108"



### RAG with LLAMA 3.2

In [26]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


In [27]:
prompt = hub.pull("rlm/rag-prompt")

In [28]:
 prompt = """
     You are an assistant that responds for question-answering tasks. Use the following pieces of retrieved context to answer the question.
     If you don't know the answer, just say that you don't know.
     Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
     Answer in spanish.
     Question: {question}
     Context: {context}
     Answer:
 """
prompt = ChatPromptTemplate.from_template(prompt)

In [29]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# print(format_docs(docs))

In [30]:
#testeo varios modelos
#!ollama pull gemma3:12b
#!ollama pull gemma3:4b
#!ollama pull gemma2:2b
!ollama pull mistral
#!ollama pull openchat

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ff82381e2bea:   0% ▕▏ 7.9 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   2% ▕▏  67 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   3% ▕▏ 104 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   3% ▕▏ 133 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   4% ▕▏ 167 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   5% ▕▏ 214 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 237 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   6% ▕▏ 256 MB/4.1 GB                  pulling manifest 
pulling 

In [31]:
!ollama list

NAME                       ID              SIZE      MODIFIED       
mistral:latest             f974a74358d6    4.1 GB    25 seconds ago    
llama3.2:3b                a80c4f17acd5    2.0 GB    4 minutes ago     
nomic-embed-text:latest    0a109f422b47    274 MB    5 minutes ago     


In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

#gemini
#model_rag = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=api_key)

#llama3.2:3b
#model_rag = ChatOllama(model="llama3.2:3b", base_url="localhost:11434")

#gemma3:12b
#model_rag = ChatOllama(model="gemma3:12b", base_url="localhost:11434")

#gemma3:4b
#model_rag = ChatOllama(model="gemma3:4b", base_url="localhost:11434")

#gemma2:2b
#model_rag = ChatOllama(model="gemma2:2b", base_url="localhost:11434")

#mistral
model_rag = ChatOllama(model="mistral", base_url="localhost:11434")

#openchat
#model_rag = ChatOllama(model="openchat", base_url="localhost:11434")


rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model_rag
    | StrOutputParser()
)

In [36]:
# question = "what is used to gain muscle mass?"
# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"

#question = "what is used to increase mass of the Earth?"

question = "incidentes creados por Gabriel"

output = rag_chain.invoke(question)
print(output)


 - Incidentes creados por Gabriel:
     - Id incidente: 173, asunto: "Fallo en la validación del RUC emisor", fecha_incidente: "2024-02-29", estado: "En progreso"
     - Id incidente: 115, asunto: "Incompatibilidad con el certificado digital", fecha_incidente: "2024-08-21", estado: "Resuelto", prioridad: "Baja"
     - Id incidente: 165, asunto: "Incompatibilidad con el certificado digital", fecha_incidente: "2024-03-26", estado: "Abierto", prioridad: sin especificar
     - Id incidente: 109, asunto: "Comprobante no enviado correctamente a DGI", fecha_incidente: "2023-07-07", estado: "En progreso", prioridad: sin especificar
     - Id incidente: 159, asunto: "Comprobante no enviado correctamente a DGI", fecha_incidente: "2024-08-21", estado: "Resuelto", prioridad: "Baja"
     - Id incidente: último, asunto: "Firma", fecha_incidente: "2025-04-01", estado: "Pendiente", prioridad: sin especificar


In [ ]:
import gradio as gr # oh yeah!

In [ ]:
# Let's create a call that streams back results

def chatbot(question):

    docs = retriever.invoke(question)

    rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model_rag
    | StrOutputParser())

    output = rag_chain.invoke(question)

    return output

    # result = ""
    # for chunk in stream:
    #     result += chunk.choices[0].delta.content or ""
    #     yield result


In [ ]:
view = gr.Interface(
    fn=chatbot,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=20)],
    flagging_mode="never"
)

gr.themes.Ocean()

view.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5ab2f8924dde4123a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
